In [1]:
from spacer import Spacer


In [3]:
spacer = Spacer()

soup = spacer.get_posts('nasas-perseverance-is-exploring-mars-come-watch-updates-with-us.37226', 1)

In [17]:
boxes = soup.find_all("div", class_="message-userContent lbContainer js-lbContainer")
posts = []
for box in boxes:
    meta = box['data-lb-caption-desc'].split(' · ')
    user, timestamp = meta[0], meta[1]
    post = box.text
    posts.append(
        {
            'user': user,
            'time': timestamp,
            'post': post
        }
    )